In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Create levels with multiple starting conformations

In [2]:
import copy

min_rmsd = 0
max_rmsd = 1
stop = False
level = 0
rmsds_dict = {}

while stop == False:
    stop = True
    for i in range(64):
        mol1 = copy.deepcopy(mol)
        mol1.dropFrames(keep=[i])


        mol1.align('name CA', refmol=ref_mol)
        rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
        rmsd = rmsd_proj.project(mol1)

        if min_rmsd <= rmsd <= max_rmsd:
            rmsds_dict['level'+str(level + 1)] = rmsd.item()
            print(rmsd)
            min_rmsd += 1
            max_rmsd += 1
            stop = False       
            path = '/workspace7/torchmd-AD/train_curriculum/cln/levels/level' + str(level)
            #os.mkdir(path)
            mol1.write(os.path.join(path, 'chignolin_cln025.xtc' ))
            level += 1
    
    

NameError: name 'mol' is not defined

In [96]:
import pandas as pd
pd.DataFrame.from_dict(rmsds_dict, orient='index', columns = ['starting rmsd'])

,starting rmsd
level1,0.873917
level2,1.463428
level3,2.858741
level4,3.779713
level5,4.009621
level6,5.704338
level7,6.318629


In [21]:
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/psf/chignolin_cln025.psf')
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/xtc/chignolin_cln025.xtc')

NameError: name 'mol_curriculum' is not defined

## Create levels with multiple starting conformations

In [54]:
# Reference Molecule
psf = f'../data/bba/bba_ca_top_dih.psf'
ref_pdb = f'../data/bba/bba_ca.pdb'
ref_mol = Molecule(psf)
ref_mol.read(ref_pdb)

ref_mol.filter('name CA')

# Read the array with trajectory data

mol = Molecule(psf)
arr = np.load('/workspace2/fast_folders_cgnet/multiprotein/data_dih/bba_ca_coords.npy')[:100000]
mol.coords = np.moveaxis(arr, 0, -1)

ref_mol.write('/workspace7/torchmd-AD/train_curriculum/fastfolders/level0/ground_truth/bba_ca.pdb')

In [8]:
# Compute the rmsd of each frame against the reference structure
mol.align('name CA', refmol=ref_mol)
rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
rmsd = rmsd_proj.project(mol)

NameError: name 'mol' is not defined

In [72]:
mol.coords.shape
#arr.shape

(28, 3, 100000)

In [83]:
arr = np.load('/shared/carles/repo/torchmd-exp/scripts/trainings/tests/test_fastfolders/save_tica/chignolin_cln025.npy')
coords = np.moveaxis(arr, 0, -1)

In [84]:
coords.shape

(10, 3, 5920)

In [154]:
# Sample frames for different levels and save them in levels folders
from statistics import mean
n_levels = int(1 // 1)
min_rmsd = 0
max_rmsd = 3

path = '/workspace7/torchmd-AD/train_curriculum/fastfolders/level0/init_states/'
mol_frames = Molecule(psf)

for n in range(n_levels):
    
    indexes = np.where(np.logical_and(rmsd>=min_rmsd, rmsd<=max_rmsd))[0]
    frames = arr[indexes, :, :][:200]
    
    print(mean(rmsd[indexes]))
    print(frames.shape)
    
    directory = path
    #os.mkdir(directory)
    
    mol_frames.coords = np.moveaxis(frames, 0, -1)
    mol_frames.write(path + 'bba_ca.xtc')
    
    #np.save(os.path.join(directory, 'ww_ca.npy'), frames)
    #min_rmsd += 2
    #max_rmsd += 2
    
    

2.8952222
(21, 28, 3)
